In [5]:
import os
# # Confirm the current working directory
os.chdir("..\\")
os.getcwd()


'C:\\Users\\A547184\\Git\\Repos\\python-cronicals\\ML\\halfrnn'

In [6]:
import logging
from pathlib import Path
# from dotenv import find_dotenv, load_dotenv
import os
import pandas as pd
import numpy as np
import tensorflow as tf
print("Tensorflow version:", tf.__version__)
from tensorflow import keras

Tensorflow version: 1.7.0


In [13]:
SEQUENCE_LENGTH = 25
VACAB_FEATURES = 50
VOCAB_SIZE = 3443 # 60064 # 3575 # 3443


In [14]:
train_tokens_file = 'data/processed/train/reviews_523_tokens_25.npy'  # Check SEQUENCE_LENGTH flag
test_tokens_file = 'data/processed/test/test_reviews_556_tokens_25.npy'

logger = logging.getLogger(__name__)
logger.info('Loading data for training')

# Load the file
logger.info('Load Train file {0!s}'.format(train_tokens_file))
train_array = np.load(train_tokens_file)
logger.info('Shape of Train {0!s}'.format(train_array.shape))


In [15]:
review_text_array = train_array

In [40]:
logger = logging.getLogger('__prepare_training_data__')
# Shuffle the array
logger.info('Shuffle the array')
np.random.shuffle(review_text_array)

# Prepare tokenizer
logger.info('Prepare tokenizer')
t = keras.preprocessing.text.Tokenizer()
t.fit_on_texts(review_text_array[0:, 0])
text_seq = t.texts_to_sequences(review_text_array[0:, 0])
vocab_size = len(t.word_index) + 1
print('Vocabulary size : {}'.format(vocab_size))

assert(len(t.word_index) + 1 <= VOCAB_SIZE)
assert(max([len(iner_list) for iner_list in text_seq]) == SEQUENCE_LENGTH)

# Padding Sequences
input_seq_padded = keras.preprocessing.sequence.pad_sequences(text_seq, maxlen=SEQUENCE_LENGTH)
print(input_seq_padded.shape)
print(type(input_seq_padded))
print(input_seq_padded[0:2])

# Get the maximum element from a Numpy array
max_element = np.amax(input_seq_padded)
print(max_element)

# # Input normalization
# input_seq_normalized = input_seq_padded / max_element
input_seq_normalized = np.asarray(input_seq_padded, dtype=np.float)

# Output data type conversion
# type(input_seq_normalized[0:1, 0][0])
output = np.asarray(review_text_array[0:, 1], dtype=np.float)
output -= 1
output_y = keras.utils.to_categorical(output, num_classes=10)

Vocabulary size : 3443
(523, 25)
<class 'numpy.ndarray'>
[[   0    0    0    0    0    0    0    0    0   65    1   48   59  328
     7 1354 1355   20    8  230  271  329   20  143  562]
 [   0    0  105  811 1356  407 1357  812  813   13   20   36  126   38
    94  105  408  163  814  563  815  816   94   75  164]]
3442


In [41]:
print(output_y[0:10,])

[[0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]]


In [42]:
print(output[0:10,])

[7. 0. 0. 9. 8. 7. 1. 8. 7. 9.]


In [43]:
np.unique(output)

array([0., 1., 2., 3., 6., 7., 8., 9.])

In [44]:
output.shape

(523,)